In [1]:
# Cell 1: run for dependencies
# %pip install --upgrade pip
# %pip install "ultralytics>=8.3.0" opencv-python pillow scikit-learn albumentations


In [2]:
# Cell 2: imports + hard fences
import os, shutil, random
from pathlib import Path
import cv2, numpy as np
from PIL import Image
import albumentations as A
from sklearn.model_selection import StratifiedShuffleSplit

PROJECT_ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
ROOT = PROJECT_ROOT
os.environ["WANDB_DISABLED"] = "true"
os.environ["MLFLOW_DISABLE_ENV"] = "true"
os.environ["ULTRALYTICS_SETTINGS_RESET"] = "1"


In [3]:
# Cell 3: paths + params
RAW_DATA  = ROOT / "data"
YOLO_ROOT = ROOT / "yolo_data"
RUNS_DIR  = YOLO_ROOT / "runs"

IM_TRAIN = YOLO_ROOT / "images" / "train"
IM_VAL   = YOLO_ROOT / "images" / "val"
LB_TRAIN = YOLO_ROOT / "labels" / "train"
LB_VAL   = YOLO_ROOT / "labels" / "val"
YOLO_YAML = YOLO_ROOT / "data.yaml"

VAL_SPLIT = 0.15
AUG_MULTIPLIER = 5
RANDOM_SEED = 42
random.seed(RANDOM_SEED); np.random.seed(RANDOM_SEED)

CENTER_BOX_FRAC = 0.30

MODEL_NAME = "yolov8n.pt"
IMG_SIZE   = 416
EPOCHS     = 40
BATCH      = 2
WORKERS    = 0

assert RAW_DATA.exists(), f"Missing: {RAW_DATA}"
class_names = sorted([d.name for d in RAW_DATA.iterdir() if d.is_dir()])
assert len(class_names) == 4, f"Expected 4 classes, got {len(class_names)}: {class_names}"
class_to_idx = {c:i for i,c in enumerate(class_names)}
print("Classes:", class_names)

VALID_EXTS = {".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff"}


Classes: ['Capacitor', 'IC', 'Processor', 'Tan_Cap']


In [4]:
# Cell 4: io helpers
def list_images():
    items = []
    for cls in class_names:
        for p in (RAW_DATA/cls).rglob("*"):
            if p.suffix.lower() in VALID_EXTS:
                items.append((p, class_to_idx[cls]))
    return items

def load_rgb(path):
    try:
        im = Image.open(path).convert("RGB")
        return np.array(im)
    except Exception as e:
        print(f"[skip] {path} ({e})"); return None

def find_largest_box(img):
    h, w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 31, 5)
    k = np.ones((3,3), np.uint8)
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, k, iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, k, iterations=2)
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return None
    area_min = 0.02 * (w*h)
    cands = []
    for c in cnts:
        x,y,ww,hh = cv2.boundingRect(c)
        area = ww*hh
        if area < area_min: continue
        ar = ww / max(1, hh)
        if ar < 0.2 or ar > 5.0: continue
        cands.append((x,y,x+ww,y+hh, area))
    if not cands: return None
    x1,y1,x2,y2,_ = max(cands, key=lambda t:t[4])
    px, py = int(0.02*w), int(0.02*h)
    x1 = max(0, x1-px); y1 = max(0, y1-py)
    x2 = min(w-1, x2+px); y2 = min(h-1, y2+py)
    return (x1,y1,x2,y2)

def yolo_line_from_box(cls_id, box, w, h):
    x1,y1,x2,y2 = box
    bw, bh = (x2-x1), (y2-y1)
    cx, cy = x1 + bw/2, y1 + bh/2
    return f"{cls_id} {cx/w:.6f} {cy/h:.6f} {bw/w:.6f} {bh/h:.6f}\n"


In [5]:
# Cell 5: split + dirs
all_items = list_images()
assert len(all_items) > 0, "No images found under ./data/<class>."
print(f"Found {len(all_items)} images")

y = [cid for _, cid in all_items]
idx = np.arange(len(all_items))
sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SPLIT, random_state=RANDOM_SEED)
train_idx, val_idx = next(sss.split(idx, y))
train_items = [all_items[i] for i in train_idx]
val_items   = [all_items[i] for i in val_idx]
print(f"Train: {len(train_items)} | Val: {len(val_items)}")

# SAFE: keep runs/ (weights) intact; only reset dataset folders
for p in [IM_TRAIN, IM_VAL, LB_TRAIN, LB_VAL]:
    if p.parent.parent.exists() and p.parent.parent != YOLO_ROOT:
        pass  # just being defensive
# recreate images/labels trees
for p in [IM_TRAIN, IM_VAL, LB_TRAIN, LB_VAL]:
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

RUNS_DIR.mkdir(parents=True, exist_ok=True)  # keep existing runs

IM_TRAIN.mkdir(parents=True, exist_ok=True); IM_VAL.mkdir(parents=True, exist_ok=True)
LB_TRAIN.mkdir(parents=True, exist_ok=True); LB_VAL.mkdir(parents=True, exist_ok=True)
RUNS_DIR.mkdir(parents=True, exist_ok=True)


Found 414 images
Train: 351 | Val: 63


In [6]:
# Cell 6: write base split
def write_split(items, im_dst, lb_dst):
    kept = 0
    for src_path, cls_id in items:
        img = load_rgb(src_path)
        if img is None: continue
        h, w = img.shape[:2]
        box = find_largest_box(img)
        if box is None:
            bw, bh = int(0.5*w), int(0.5*h)
            x1 = (w-bw)//2; y1 = (h-bh)//2
            box = (x1,y1,x1+bw,y1+bh)
        out_img = im_dst / src_path.name
        Image.fromarray(img).save(out_img)
        yolo_line = yolo_line_from_box(cls_id, box, w, h)
        (lb_dst / (out_img.stem + ".txt")).write_text(yolo_line)
        kept += 1
    print(f"Wrote {kept} -> {im_dst}")

write_split(train_items, IM_TRAIN, LB_TRAIN)
write_split(val_items,   IM_VAL,   LB_VAL)


C:\Users\armaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\PIL\Image.py:3570: UserWarning: image file could not be identified because AVIF support not installed
  warnings.warn(message)


[skip] d:\Coding\Projects\Classifier\Waste_Classifier\data\Processor\cpu_253658-1870 (3).jpg (cannot identify image file 'd:\\Coding\\Projects\\Classifier\\Waste_Classifier\\data\\Processor\\cpu_253658-1870 (3).jpg')
[skip] d:\Coding\Projects\Classifier\Waste_Classifier\data\Processor\computer-microprocessor-cpu-close-up-isolated-white-backgrou.jpg (cannot identify image file 'd:\\Coding\\Projects\\Classifier\\Waste_Classifier\\data\\Processor\\computer-microprocessor-cpu-close-up-isolated-white-backgrou.jpg')
[skip] d:\Coding\Projects\Classifier\Waste_Classifier\data\Processor\single-microchip-isolated-white-background_941600-88404(1).jpg (cannot identify image file 'd:\\Coding\\Projects\\Classifier\\Waste_Classifier\\data\\Processor\\single-microchip-isolated-white-background_941600-88404(1).jpg')
[skip] d:\Coding\Projects\Classifier\Waste_Classifier\data\Processor\high-angle-view-computer-chip-against-white-background_10489 (1).jpg (cannot identify image file 'd:\\Coding\\Projects\\

In [7]:
# Cell 7: aug pipeline + helpers
AUG_PIPE = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.6),
    A.HueSaturationValue(hue_shift_limit=8, sat_shift_limit=20, val_shift_limit=15, p=0.4),
    A.GaussNoise(var_limit=(5.0, 25.0), p=0.2),
    A.MotionBlur(blur_limit=5, p=0.2),
    A.Affine(scale=(0.85, 1.15), translate_percent=(0.0, 0.05), rotate=(-10, 10),
             shear=(-5, 5), mode=cv2.BORDER_REFLECT_101, p=0.5),
    A.Resize(height=int(IMG_SIZE), width=int(IMG_SIZE), interpolation=cv2.INTER_LINEAR),
], bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"], min_visibility=0.4))

def yolo_to_voc_px(cx, cy, bw, bh, w, h):
    x1 = (cx - bw/2) * w; y1 = (cy - bh/2) * h
    x2 = (cx + bw/2) * w; y2 = (cy + bh/2) * h
    return [x1, y1, x2, y2]

def voc_px_to_yolo(x1, y1, x2, y2, w, h):
    bw = max(1.0, x2 - x1); bh = max(1.0, y2 - y1)
    cx = x1 + bw/2; cy = y1 + bh/2
    return [cx / w, cy / h, bw / w, bh / h]

def clamp_voc(x1, y1, x2, y2, w, h):
    x1 = max(0.0, min(float(w - 1), x1))
    y1 = max(0.0, min(float(h - 1), y1))
    x2 = max(0.0, min(float(w - 1), x2))
    y2 = max(0.0, min(float(h - 1), y2))
    if x2 <= x1: x2 = min(w - 1.0, x1 + 1.0)
    if y2 <= y1: y2 = min(h - 1.0, y1 + 1.0)
    return [x1, y1, x2, y2]


C:\Users\armaa\AppData\Local\Temp\ipykernel_3796\732167412.py:6: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 25.0), p=0.2),
C:\Users\armaa\AppData\Local\Temp\ipykernel_3796\732167412.py:8: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(scale=(0.85, 1.15), translate_percent=(0.0, 0.05), rotate=(-10, 10),


In [8]:
# Cell 8: augment
def augment_train_set(mult=AUG_MULTIPLIER):
    ims = sorted([p for p in IM_TRAIN.iterdir() if p.suffix.lower() in VALID_EXTS])
    count = 0
    for p in ims:
        lbp = LB_TRAIN / (p.stem + ".txt")
        if not lbp.exists(): continue
        img_bgr = cv2.imread(str(p))
        if img_bgr is None: continue
        img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        lines = [ln.strip().split() for ln in lbp.read_text().strip().splitlines() if ln.strip()]
        if not lines: continue
        bboxes_voc, class_labels = [], []
        for ln in lines:
            c = int(ln[0]); cx, cy, bw, bh = map(float, ln[1:5])
            x1, y1, x2, y2 = yolo_to_voc_px(cx, cy, bw, bh, w, h)
            bboxes_voc.append([x1, y1, x2, y2]); class_labels.append(c)

        for k in range(mult):
            try:
                t = AUG_PIPE(image=img, bboxes=bboxes_voc, class_labels=class_labels)
            except Exception:
                continue
            im2 = t["image"]; bb2 = t["bboxes"]; cl2 = t["class_labels"]
            if len(bb2) == 0: continue
            h2, w2 = im2.shape[:2]
            filtered = []
            for (x1, y1, x2, y2), c in zip(bb2, cl2):
                x1, y1, x2, y2 = clamp_voc(x1, y1, x2, y2, w2, h2)
                bw = x2 - x1; bh = y2 - y1
                if bw * bh < 4: continue
                filtered.append((c, x1, y1, x2, y2))
            if not filtered: continue

            out_img = IM_TRAIN / f"{p.stem}_aug{k}.jpg"
            cv2.imwrite(str(out_img), cv2.cvtColor(im2, cv2.COLOR_RGB2BGR))
            with open(LB_TRAIN / f"{p.stem}_aug{k}.txt", "w") as f:
                for c, x1, y1, x2, y2 in filtered:
                    cx, cy, bw, bh = voc_px_to_yolo(x1, y1, x2, y2, w2, h2)
                    cx = min(1.0, max(0.0, cx)); cy = min(1.0, max(0.0, cy))
                    bw = min(1.0, max(1e-6, bw)); bh = min(1.0, max(1e-6, bh))
                    f.write(f"{c} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")
            count += 1
    print(f"Augmented images written: {count}")

augment_train_set(mult=AUG_MULTIPLIER)


Augmented images written: 1540


In [9]:
# Cell 9: data.yaml
YOLO_ROOT.mkdir(parents=True, exist_ok=True)
with open(YOLO_YAML, "w") as f:
    f.write(f"path: {str(YOLO_ROOT)}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("names:\n")
    for i,n in enumerate(class_names):
        f.write(f"  {i}: {n}\n")
print(YOLO_YAML.read_text())


path: d:\Coding\Projects\Classifier\Waste_Classifier\yolo_data
train: images/train
val: images/val
names:
  0: Capacitor
  1: IC
  2: Processor
  3: Tan_Cap



In [10]:
# Cell 10: train
from ultralytics import YOLO

def train_yolo():
    model = YOLO(MODEL_NAME)
    run_name = "train_exp"
    model.train(
        data=str(YOLO_YAML),
        device="cpu",
        imgsz=IMG_SIZE,
        epochs=EPOCHS,
        batch=BATCH,
        workers=WORKERS,
        amp=False,
        mosaic=0.0,
        cache=False,
        freeze=0,
        patience=10,
        verbose=True,
        project=str(RUNS_DIR),
        name=run_name,
        exist_ok=True,
        save=True,
        save_period=-1,
        resume=False,
        seed=RANDOM_SEED
    )
    best_path = Path(getattr(model.trainer, "best", "")).resolve()
    assert best_path.is_file(), f"Best weights not found: {best_path}"
    assert RUNS_DIR.resolve() in best_path.parents, f"Outside project: {best_path}"
    print("Best model:", best_path)
    return best_path


In [11]:
# Cell 11: validate
def validate(weights_path: Path):
    model = YOLO(str(weights_path))
    run_name = "val_exp"
    val_res = model.val(
        data=str(YOLO_YAML),
        imgsz=IMG_SIZE,
        device="cpu",
        project=str(RUNS_DIR),
        name=run_name,
        exist_ok=True,
        save=False,
        plots=False
    )
    print(f"mAP50-95: {val_res.box.map:.4f} | mAP50: {val_res.box.map50:.4f}")
    for i, n in enumerate(class_names):
        ap = val_res.box.maps[i] if i < len(val_res.box.maps) else float("nan")
        print(f"{n}: AP50-95={ap:.4f}")


In [12]:
# Cell 12: helpers
def is_centered(box_xyxy, frame_w, frame_h, frac=CENTER_BOX_FRAC):
    x1,y1,x2,y2 = box_xyxy
    cx = 0.5*(x1+x2); cy = 0.5*(y1+y2)
    fx = frame_w*0.5; fy = frame_h*0.5
    hw = frame_w*frac*0.5; hh = frame_h*frac*0.5
    return (fx-hw) <= cx <= (fx+hw) and (fy-hh) <= cy <= (fy+hh)

def load_weights_or_fail(path: Path) -> Path:
    path = Path(path).resolve()
    assert path.is_file(), f"Missing weights: {path}"
    assert RUNS_DIR.resolve() in path.parents, f"Refusing non-project weights: {path}"
    return path


In [13]:
# Cell 13: predict image
def predict_image(image_path, weights_path, conf_th=0.10):
    weights_path = load_weights_or_fail(weights_path)
    mdl = YOLO(str(weights_path))
    res = mdl.predict(
        source=str(image_path),
        imgsz=IMG_SIZE,
        conf=conf_th,
        verbose=False,
        device="cpu",
        save=False, save_txt=False, save_conf=False,
        project=str(RUNS_DIR), name="predict_exp", exist_ok=True
    )
    for r in res:
        if r.boxes is None or len(r.boxes)==0:
            print("No detections."); continue
        for b in r.boxes:
            cid = int(b.cls.item()); conf = float(b.conf.item())
            xyxy = list(map(int, b.xyxy.squeeze().cpu().numpy().tolist()))
            print(f"{class_names[cid]} @ {conf:.2f} -> {xyxy}")
    for r in res:
        r.show()


In [14]:
# Cell 14: live webcam
def live_video_classify(weights_path, cam_index=0, conf_th=0.15):
    from collections import deque
    weights_path = load_weights_or_fail(weights_path)
    mdl = YOLO(str(weights_path))
    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        print("Could not open webcam."); return
    history = deque(maxlen=5)
    try:
        while True:
            ok, frame = cap.read()
            if not ok: break
            h, w = frame.shape[:2]
            res = mdl.predict(
                source=frame,
                imgsz=IMG_SIZE,
                conf=conf_th,
                verbose=False,
                device="cpu",
                save=False, save_txt=False, save_conf=False,
                project=str(RUNS_DIR), name="live_exp", exist_ok=True
            )
            preds = []
            for r in res:
                if r.boxes is None: continue
                for b in r.boxes:
                    cid = int(b.cls.item()); conf = float(b.conf.item())
                    xyxy = b.xyxy.squeeze().cpu().numpy().tolist()
                    preds.append((cid, conf, xyxy))

            fx, fy = int(w*0.5), int(h*0.5)
            hw, hh = int(w*CENTER_BOX_FRAC*0.5), int(h*CENTER_BOX_FRAC*0.5)
            cv2.rectangle(frame, (fx-hw, fy-hh), (fx+hw, fy+hh), (128,128,128), 1)

            any_yes = False
            for (cid, conf, xyxy) in preds:
                x1,y1,x2,y2 = map(int, xyxy)
                centered = is_centered(xyxy, w, h, frac=CENTER_BOX_FRAC)
                any_yes |= centered
                color = (0,255,0) if centered else (0,165,255)
                cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
                label = f"{class_names[cid]} {conf:.2f}"
                cv2.putText(frame, label, (x1, max(20,y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            history.append(1 if any_yes else 0)
            stable_yes = sum(history) >= 3
            status = "YES" if stable_yes else "NO"
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                        (0,255,0) if stable_yes else (0,0,255), 2)

            cv2.imshow("Live Detection (q to quit)", frame)
            if (cv2.waitKey(1) & 0xFF) == ord('q'): break
    finally:
        cap.release(); cv2.destroyAllWindows()


In [15]:
best = train_yolo()




New https://pypi.org/project/ultralytics/8.3.222 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.210  Python-3.13.9 torch-2.8.0+cu126 CPU (Intel Core i9-14900HX)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=d:\Coding\Projects\Classifier\Waste_Classifier\yolo_data\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train_

In [16]:
validate(best)

Ultralytics 8.3.210  Python-3.13.9 torch-2.8.0+cu126 CPU (Intel Core i9-14900HX)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 914.0889.3 MB/s, size: 61.0 KB)
val: Scanning D:\Coding\Projects\Classifier\Waste_Classifier\yolo_data\labels\val.cache... 51 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 51/51 127.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.1it/s 1.9s0.9s
                   all         51         51      0.762      0.856      0.858      0.726
             Capacitor         27         27      0.791      0.852      0.869       0.74
                    IC          7          7      0.749      0.853      0.696      0.384
             Processor         10         10      0.782      0.718      0.873      0.817
               Tan_Cap          7          7      0.724          1      0.995      0.963
Speed: 0.5ms prep

In [ ]:
live_video_classify(best)